<a href="https://colab.research.google.com/github/daisuke00001/01tuning/blob/main/notebooks/TinySwallow_1_5B_Alpaca_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TinySwallow-1.5B-Instruct LoRAチューニング実装
#### Google Colab T4 GPU環境用


In [19]:
# 1. 自動リロード設定（初回のみ）
%load_ext autoreload
%autoreload 2

# 2. 更新があるたびに実行
!git pull origin main

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 634 bytes | 634.00 KiB/s, done.
From https://github.com/daisuke00001/01tuning
 * branch            main       -> FETCH_HEAD
   ecc576c..ba18fc6  main       -> origin/main
Updating ecc576c..ba18fc6
Fast-forward
 src/inference_utils.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)


In [2]:
# 実行環境の確認とgithubリポジトリのクローン
import os
import subprocess
import sys

In [3]:
def setup_colab_environment():
    """Google Colab環境をセットアップ"""
    print("🔧 Google Colab 環境をセットアップ中...")

    # GPU確認
    if not os.path.exists('/opt/bin/nvidia-smi'):
       print("  GPU環境が検出されません。ランタイムタイプをGPUに変更してください。")
       return False

    # リポジトリクローン
    repo_url = "https://github.com/daisuke00001/01tuning.git"
    if os.path.exists("01tuning"):
        print("📁 リポジトリが既に存在します。最新版に更新中...")
        subprocess.run(["git", "-C", "01tuning", "pull"], check=True)
    else:
        print(f"📁 リポジトリをクローン中: {repo_url}")
        subprocess.run(["git", "clone", repo_url], check=True)

    # 作業ディレクトリに移動
    os.chdir("01tuning")

    if "/content/01tuning/src" not in sys.path:
        sys.path.append("/content/01tuning/src")

    print("✅ 環境セットアップ完了")
    return True

# セットアップ実行
setup_colab_environment()


🔧 Google Colab 環境をセットアップ中...
📁 リポジトリが既に存在します。最新版に更新中...
✅ 環境セットアップ完了


True

## ライブラリインストール

In [4]:
# Google Colab専用のライブラリインストール
%%capture
# Unslothとその依存関係
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer
!pip install --no-deps unsloth

# ライブラリエラー対応
!pip install xformers
!pip uninstall torchvision -y
!pip install torchvision --no-cache-dir

# その他の依存関係
!pip install PyYAML

print("📦 ライブラリインストール完了")

## 設定の読み込み

In [5]:
import yaml
from src.config import Config

# 設定ファイルの読み込み
config_path = "/content/01tuning/configs/tinyswallow_config.yaml"
config = Config.load_from_yaml(config_path)

print("⚙️  設定内容:")
print(f"  モデル: {config.model.name}")
print(f"  最大シーケンス長: {config.model.max_seq_length}")
print(f"  LoRAランク: {config.lora.r}")
print(f"  バッチサイズ: {config.training.per_device_train_batch_size}")
print(f"  最大ステップ数: {config.training.max_steps}")
print(f"  学習率: {config.training.learning_rate}")
print(f"  データセット: {config.dataset.name}")

⚙️  設定内容:
  モデル: SakanaAI/TinySwallow-1.5B-Instruct
  最大シーケンス長: 1024
  LoRAランク: 16
  バッチサイズ: 1
  最大ステップ数: 100
  学習率: 5e-05
  データセット: alpaca_japanese


## モデルの読み込みとLoRA設定

In [6]:
# Google Colabでリポジトリ更新
%cd /content
!rm -rf 01tuning
!git clone https://github.com/daisuke00001/01tuning.git
%cd 01tuning

/content
Cloning into '01tuning'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 109 (delta 50), reused 90 (delta 35), pack-reused 0 (from 0)
Receiving objects: 100% (109/109), 59.81 KiB | 3.52 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/01tuning


In [7]:
from model_utils import ModelManager
import torch

# モデル管理クラスの初期化
model_manager = ModelManager(config)

# モデルの読み込み
model, tokenizer = model_manager.load_model()

# LoRA設定の適用
model = model_manager.setup_lora()

# メモリ使用量の確認
memory_stats = model_manager.get_memory_stats()
print(f"\n💾 GPU情報:")
print(f"  GPU: {memory_stats['gpu_name']}")
print(f"  総メモリ: {memory_stats['total_memory_gb']} GB")
print(f"  使用メモリ: {memory_stats['reserved_memory_gb']} GB")
print(f"  使用率: {memory_stats['usage_percentage']}%")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.7.1+cu126)
    Python  3.11.11 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.11: Fast Qwen2 patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.7.11 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.



💾 GPU情報:
  GPU: Tesla T4
  総メモリ: 14.741 GB
  使用メモリ: 1.213 GB
  使用率: 8.2%


## データの準備

In [8]:
# データ処理クラスを使用してデータセットを準備
from data_processing import DataProcessor

# データ処理クラスの初期化
data_processor = DataProcessor(config, tokenizer)

# データセットの読み込みと前処理
dataset = data_processor.load_and_process_dataset()

print(f"\n📊 データセット情報:")
print(f"  総サンプル数: {len(dataset):,}件")
print(f"  テキストフィールド: {config.data.text_field}")

# サンプルデータの確認
print(f"\n📝 サンプルデータ（先頭200文字）:")
print(dataset[0]['text'][:200] + "...")

Map (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]


📊 データセット情報:
  総サンプル数: 50件
  テキストフィールド: text

📝 サンプルデータ（先頭200文字）:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

        ### Instruction:
        以下の文章を要約...


## トレーニングの準備と実行

In [12]:
# トレーニング管理クラスを使用してトレーニングを実行
from src.training_utils import TrainingManager

# トレーニング管理クラスの初期化
training_manager = TrainingManager(config)

# トレーナーの作成
trainer = training_manager.create_trainer(model, tokenizer, dataset)

print("🚀 トレーニング開始...")
print("=" * 50)
print(f"  モデル: {config.model.name}")
print(f"  ステップ数: {config.training.max_steps}")
print(f"  バッチサイズ: {config.training.per_device_train_batch_size}")
print(f"  学習率: {config.training.learning_rate}")
print("=" * 50)

# トレーニング実行
training_stats = training_manager.train()

print("\n✅ トレーニング完了！")

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

🚀 トレーニング開始...
  モデル: SakanaAI/TinySwallow-1.5B-Instruct
  ステップ数: 100
  バッチサイズ: 1
  学習率: 5e-05


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 8 | Total steps = 100
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 18,464,768 of 1,562,179,072 (1.18% trained)


Step,Training Loss
10,1.717800
20,0.718600
30,0.238700
40,0.066800
50,0.036500
60,0.033300
70,0.027000
80,0.026400
90,0.025300
100,0.024900



✅ トレーニング完了！


## トレーニング結果を確認

In [15]:
# トレーニング結果とメモリ使用量を確認
summary = training_manager.get_training_summary()
final_memory = model_manager.get_memory_stats()

print("📊 トレーニング結果:")
print("-" * 40)
print(f"⏱️  実行時間: {summary['train_runtime_seconds']} 秒 ({summary['train_runtime_minutes']} 分)")
print(f"📈 サンプル/秒: {summary['train_samples_per_second']:.2f}")
print(f"🔄 ステップ/秒: {summary['train_steps_per_second']:.2f}")
print(f"📉 最終Loss: {summary['train_loss']:.4f}")

print(f"\n💾 最終メモリ使用量:")
print(f"  使用メモリ: {final_memory['reserved_memory_gb']} GB")
print(f"  使用率: {final_memory['usage_percentage']}%")

📊 トレーニング結果:
----------------------------------------
⏱️  実行時間: 221.9 秒 (3.7 分)
📈 サンプル/秒: 1.80
🔄 ステップ/秒: 0.45
📉 最終Loss: 0.2915

💾 最終メモリ使用量:
  使用メモリ: 1.557 GB
  使用率: 10.6%


## 推論テスト

In [20]:
# 推論管理クラスを使用してテスト生成
from src.inference_utils import InferenceManager

# 推論管理クラスの初期化
inference_manager = InferenceManager(model, tokenizer)

print("🧪 推論テスト実行中...")

# Alpacaフォーマットでのテスト
test_cases = [
    {
        "instruction": "フィボナッチ数列を続けてください。",
        "input": "1, 1, 2, 3, 5, 8"
    },
    {
        "instruction": "首都を教えてください。",
        "input": "日本"
    },
    {
        "instruction": "次の文章を要約してください。",
        "input": "人工知能技術は急速に発展しており、様々な分野で活用されています。特に大規模言語モデルは、文章生成や翻訳、質問応答など多くのタスクで高い性能を示しています。"
    }
]

print("\n" + "=" * 60)
print("📤 生成結果:")
print("=" * 60)

for i, test_case in enumerate(test_cases, 1):
    print(f"\n【テスト {i}】")
    print(f"指示: {test_case['instruction']}")
    print(f"入力: {test_case['input']}")

    response = inference_manager.test_alpaca_format(
        test_case['instruction'],
        test_case['input']
    )

    # 応答部分のみを抽出
    if "### Response:" in response:
        response_part = response.split("### Response:")[-1].strip()
        print(f"応答: {response_part[:200]}...")  # 先頭200文字のみ表示
    else:
        print(f"応答: {response[:200]}...")

    print("-" * 40)

🧪 推論テスト実行中...

📤 生成結果:

【テスト 1】
指示: フィボナッチ数列を続けてください。
入力: 1, 1, 2, 3, 5, 8
応答: 13, 21, 34
        <|im_end|>...
----------------------------------------

【テスト 2】
指示: 首都を教えてください。
入力: 日本
応答: 東京です。
        <|im_end|>...
----------------------------------------

【テスト 3】
指示: 次の文章を要約してください。
入力: 人工知能技術は急速に発展しており、様々な分野で活用されています。特に大規模言語モデルは、文章生成や翻訳、質問応答など多くのタスクで高い性能を示しています。
応答: AI技術は進化し続け、多岐にわたる分野で利用されています。特に、大規模な言語モデルは様々なタスクにおいて優れたパフォーマンスを発揮します。
        <|im_end|>...
----------------------------------------


## モデルの保存

In [22]:
# 学習済みモデルを保存
save_path = f"models/tinyswallow_lora_{config.training.max_steps}steps"  # <- 本来であればベストモデルを保存するべき

print(f"💾 モデルを保存中: {save_path}")

# ディレクトリ作成
os.makedirs(save_path, exist_ok=True)

# モデル保存
model_manager.save_model(save_path)

# 設定ファイルも一緒に保存
config.save_yaml(f"{save_path}/config.yaml")

print("✅ モデル保存完了")
print(f"📁 保存先: {save_path}/")

💾 モデルを保存中: models/tinyswallow_lora_100steps
✅ モデル保存完了
📁 保存先: models/tinyswallow_lora_100steps/


## 保存オプション

In [ ]:
# 16bit マージ済みモデル（高品質、サイズ大）
print("💾 16bit マージ済みモデルを保存中...")
model.save_pretrained_merged(f"{save_path}_merged_16bit", tokenizer, save_method="merged_16bit")

# 4bit マージ済みモデル（軽量）
print("💾 4bit マージ済みモデルを保存中...")
model.save_pretrained_merged(f"{save_path}_merged_4bit", tokenizer, save_method="merged_4bit")

# GGUF形式（CPU推論用）
print("💾 GGUF形式で保存中...")
model.save_pretrained_gguf(f"{save_path}_gguf", tokenizer, quantization_method="q8_0")
"""

print("ℹ️  必要に応じて上記コードのコメントアウトを解除してください")

## 保存したモデルのテスト読み込み

In [23]:
# 保存したモデルが正常に読み込めるかテスト
print("🔄 保存したモデルの読み込みテスト...")

# 新しい設定でモデル読み込み
test_config = Config.load_from_yaml(f"{save_path}/config.yaml")
test_model_manager = ModelManager(test_config)

# 保存したモデルを読み込み
test_model, test_tokenizer = test_model_manager.load_model()

# 簡単な推論テスト
test_inference = InferenceManager(test_model, test_tokenizer)

test_response = test_inference.test_alpaca_format(
    "こんにちはと日本語で挨拶してください。",
    ""
)

print("📤 読み込みテスト結果:")
print("-" * 40)
if "### Response:" in test_response:
    response_part = test_response.split("### Response:")[-1].strip()
    print(response_part[:100] + "...")
else:
    print(test_response[:100] + "...")

print("\n✅ モデル読み込みテスト成功！")


🔄 保存したモデルの読み込みテスト...
==((====))==  Unsloth 2025.7.11: Fast Qwen2 patching. Transformers: 4.54.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
📤 読み込みテスト結果:
----------------------------------------
<|im_end|>...

✅ モデル読み込みテスト成功！


## 完了とサマリー

In [24]:
# プロジェクト完了のサマリーを表示
print("\n🎉 TinySwallow-1.5B LoRAチューニングプロジェクト完了！")
print("\n" + "=" * 60)
print("📋 実行サマリー:")
print("=" * 60)
print(f"✅ モデル: {config.model.name}")
print(f"✅ データセット: {config.data.dataset_name} ({len(dataset):,}件)")
print(f"✅ トレーニング: {config.training.max_steps}ステップ完了")
print(f"✅ 実行時間: {summary['train_runtime_minutes']} 分")
print(f"✅ 最終Loss: {summary['train_loss']:.4f}")
print(f"✅ モデル保存先: {save_path}/")

print(f"\n🚀 次のステップ:")
print("1. より多くのステップでのトレーニング (max_steps=1000+)")
print("2. 特許データなど専用データセットでの学習")
print("3. ハイパーパラメータの調整")
print("4. GGUF形式での軽量化とCPU推論")
print("5. 評価指標での性能測定")

print(f"\n📚 参考リソース:")
print("- TinySwallow: https://huggingface.co/SakanaAI/TinySwallow-1.5B-instruct")
print("- Unsloth: https://github.com/unslothai/unsloth")
print("- プロジェクト: https://github.com/daisuke00001/01tuning")

print(f"\n💡 Tips:")
print("- 本格的なトレーニングはmax_steps=1000以上を推奨")
print("- メモリ不足の場合はbatch_sizeを1に削減")
print("- より良い結果にはより多くの高品質データが重要")
print("\n🎯 Happy Fine-tuning! 🎯")


🎉 TinySwallow-1.5B LoRAチューニングプロジェクト完了！

📋 実行サマリー:
✅ モデル: SakanaAI/TinySwallow-1.5B-Instruct
✅ データセット: yahma/alpaca-cleaned-ja (50件)
✅ トレーニング: 100ステップ完了
✅ 実行時間: 3.7 分
✅ 最終Loss: 0.2915
✅ モデル保存先: models/tinyswallow_lora_100steps/

🚀 次のステップ:
1. より多くのステップでのトレーニング (max_steps=1000+)
2. 特許データなど専用データセットでの学習
3. ハイパーパラメータの調整
4. GGUF形式での軽量化とCPU推論
5. 評価指標での性能測定

📚 参考リソース:
- TinySwallow: https://huggingface.co/SakanaAI/TinySwallow-1.5B-instruct
- Unsloth: https://github.com/unslothai/unsloth
- プロジェクト: https://github.com/daisuke00001/01tuning

💡 Tips:
- 本格的なトレーニングはmax_steps=1000以上を推奨
- メモリ不足の場合はbatch_sizeを1に削減
- より良い結果にはより多くの高品質データが重要

🎯 Happy Fine-tuning! 🎯
